In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://mongodb:27017/")

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("myApp") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.2') \
.getOrCreate()

In [4]:
influenza = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://mongodb:27017/epidemics.influenza").load()

In [6]:
#influenza.show()

In [7]:
influenza.createOrReplaceTempView("influenza")

In [10]:
influenza = spark.sql("SELECT _id, country, year, week, dateds AS week_start_date, datefs AS week_end_date,\
type_A_H1 AS AH1, type_A_H1N1 AS AH1N12009, type_A_H3 AS AH3, type_A_H5 AS AH5, type_A_NS ANOTSUBTYPED,\
type_A_total AS INF_A, type_B_Yamata AS BYAMAGATA, type_B_Victoria AS BVICTORIA, type_B_ND AS BNOTDETERMINED,\
type_B_total AS INF_B, total_positives AS ALL_INF, total_negatives AS ALL_INF2 FROM influenza")
#influenza.show()

In [12]:
influenza.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics2.epidemics_influen").save()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o83.save

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o45.save

In [7]:
influenza.count()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37683)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.z

Py4JError: An error occurred while calling o41.count

In [7]:
def slice_generator(df, chunk_size=10000):
    current_row = 0
    total_rows = df.shape[0]
    while current_row < total_rows:
        yield df[current_row:current_row + chunk_size]
        current_row += chunk_size

In [11]:
mydb = client["epidemics2"]
mycol = mydb["epidemics_influenza"]

In [13]:
for df_chunk in slice_generator(influenza):
    records = df_chunk.to_dict(orient='records')
    mycol.insert_many(records)

AttributeError: 'DataFrame' object has no attribute 'shape'